# 自然语言处理搜索引擎

我们使用jieba中的ChineseAnalyzer来处理中文的文本；用whoosh这个包来建立索引。

In [32]:
from jieba.analyse.analyzer import ChineseAnalyzer
from whoosh.fields import *
import os.path
from whoosh.index import create_in
import os
from whoosh.qparser import QueryParser
from whoosh import scoring
from whoosh import qparser

创建中文分词分析器

In [33]:
analyzer = ChineseAnalyzer()

创建索引项

In [34]:
schema = Schema(title = TEXT(stored = True) ,content = TEXT(stored = True,analyzer= analyzer))

创建倒排索引，创建文件夹存放我们要生成的索引，然后打开文件材料为材料建立索引。

In [35]:
index = create_in('.\data\index', schema)
writer = index.writer()
source_dir = '.\data\page'

#get file name
file_list = os.listdir(source_dir)

words = set()
for file_name in file_list:
    content = ''
    for line in open(source_dir + '\\' + file_name, encoding='utf-8', errors='ignore').readlines():
        content += line
    for i in  analyzer(content):
        words.add(i.text)
    writer.add_document(title = source_dir + '\\' + file_name, content = content)
writer.commit()

检索测试，在显示文档的时候，我们使用TF_IDF进行排序显示。

**Test 1**

In [36]:
searcher = index.searcher()
parser = QueryParser('content', schema=index.schema)

TF_IDF = index.searcher(weighting=scoring.TF_IDF())

keyword = '美利坚合众国'
q = parser.parse(keyword)
results = TF_IDF.search(q, limit = 5)
i = 1
for hit in results:
    print('Rank: ', i)
    print('Path: ', hit['title'])
    print('Content', hit.highlights("content"))
    i += 1

Rank:  1
Path:  .\data\page\22.txt
Content 任大陆军总司令，发表《独立宣言》，宣布<b class="match term0">美利</b>坚<b class="match term1">合众</b>国正式成立。
 
1783年独立战争...在世界的主权国家中又诞生一个新的独立国家——<b class="match term0">美利</b>坚<b class="match term1">合众</b>国；顶冠内有13颗白色五角星...武力。鹰嘴叼着的黄色绶带上用拉丁文写着“<b class="match term1">合众</b>为一”，意为<b class="match term0">美利</b>坚<b class="match term1">合众</b>国由很多州组成，是一个完整的国家。
美国
Rank:  2
Path:  .\data\page\62.txt
Content 费城(7张)






宾大坐落于<b class="match term0">美利</b>坚<b class="match term1">合众</b>国的摇篮、美国第四大城市费城
Rank:  3
Path:  .\data\page\98.txt
Content 约翰逊曾经参加过第一次大陆会议并且被选为<b class="match term0">美利</b>坚<b class="match term1">合众</b>国制宪会议代表。十八世


**Test 2**

In [37]:
keyword = '中华人民共和国'
q = parser.parse(keyword)
results = TF_IDF.search(q, limit = 5)
i = 1
for hit in results:
    print('Rank: ', i)
    print('Path: ', hit['title'])
    print('Content', hit.highlights("content"))
    i += 1

Rank:  1
Path:  .\data\page\176.txt
Content 世纪初辛亥革命后，君主政体退出历史舞台，<b class="match term0">共和</b>政体建立。1949年<b class="match term1">中华人民共和国</b>成立后，在中国大陆建立了<b class="match term2">人民</b>代表大会制度的政体。
中国疆域...月1日在北京天安门广场举行开国大典，中央<b class="match term2">人民</b>政府主席毛泽东宣告：<b class="match term1">中华人民共和国</b>正式成立。
建国之初，中国政府...年10月是“文化大革命”的十年，国家和<b class="match term2">人民</b>遭受了建国以来最严重的挫折和损失。
<b class="match term1">中华人民共和国</b>主席习近平



1976年10月
Rank:  2
Path:  .\data\page\170.txt
Content 部长
刘延东
中共十八届中央政治局委员、<b class="match term3">中华人民共和</b>国国务院副总理
杜青林...管理局局长
梁国庆
第十五届中央候补委员，<b class="match term1">中华人民共和国</b>最高<b class="match term2">人民</b>检察院常务副检察...副检察长（正部级）、一级大检查官
奚晓明
<b class="match term1">中华人民共和国</b>最高<b class="match term2">人民</b>法院副院长
孙谦
<b class="match term1">中华人民共和国</b>最高<b class="match term2">人民</b>检察院副检察长
Rank:  3
Path:  .\data\page\79.txt
Content 司法规范成立的有限责任公司或国际商业公司。
《<b class="match term1">中华人民共和国</b>公司法》
最高<b class="match term2">人民</b>法院关于适用《<b class="match term1">中华人民共和国</b>公司法》若干问题的规定...公司
最高<b 

下面在搜索的功能基础上加上拼写检查功能。

In [31]:
keyword = '美利坚合国'
without_word = False
isuse = True

query = qparser.QueryParser('content', index.schema)
q = query.parse(keyword)
print('keyword: ', keyword)
if('content', keyword) not in q.all_terms():
    correct_words = TF_IDF.corrector('content')
    print('Do you mean?')
    for i, suggest in enumerate(correct_words.suggest(keyword, limit = 5)):
        print('{}.{}'.format(i+1, suggest))
    
keyword = correct_words.suggest(keyword, limit = 5)[0]
print('\nUse \'{}\' to search'.format(keyword))
q = query.parse(keyword)
results = TF_IDF.search(q, limit = 5)

i = 1
for hit in results:
    print('Rank: ', i)
    print('Path: ', hit['title'])
    print('Content', hit.highlights("content"))
    i += 1

keyword:  美利坚合国
Do you mean?
1.美利坚
2.美利坚合众国

Use '美利坚' to search
Rank:  1
Path:  .\data\page\22.txt
Content <b class="match term0">美利</b>坚合众国（英语：United...并取得胜利的13个州，50颗五角星代表<b class="match term0">美利</b>坚合众国的州数。
1818年美国...在世界的主权国家中又诞生一个新的独立国家——<b class="match term0">美利</b>坚合众国；顶冠内有13颗白色
Rank:  2
Path:  .\data\page\44.txt
Content 必须具有美国的“国民精神”，要每时每刻为<b class="match term1">美利坚</b>的强大而自豪并为此尽责，还把不断涌入的新
Rank:  3
Path:  .\data\page\62.txt
Content 费城(7张)






宾大坐落于<b class="match term0">美利</b>坚合众国的摇篮、美国第四大城
Rank:  4
Path:  .\data\page\98.txt
Content 约翰逊曾经参加过第一次大陆会议并且被选为<b class="match term0">美利</b>坚合众国制宪会议
